In [85]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import random
import math
import time
from hungarian_matching import hungarian_matching

In [86]:
# Lade die CSV-Dateien mit Personen, Teilaufgaben und Projekten
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [87]:
# Wandle die Kompetenzen in eine Liste um, um Filterung zu erleichtern
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(r",\s*")

print(f"Loaded: {len(personen_df)} people, {len(teilaufgaben_df)} tasks")
print(f"Total workload: {teilaufgaben_df['aufwand'].sum():.1f} hours")

Loaded: 150 people, 8 tasks
Total workload: 19.5 hours


In [88]:
# Belegungstracker: Speichert die bereits verplante Kapazität je Person und Monat
belegungen = {}

In [89]:
# Liste zur Speicherung der Zuweisungsergebnisse
matching_ergebnisse = []

In [90]:
# Run Hungarian Algorithm
print("\n" + "="*50)
print("HUNGARIAN ALGORITHM MATCHING")
print("="*50)


HUNGARIAN ALGORITHM MATCHING


In [96]:
hungarian_results, hungarian_time = hungarian_matching(teilaufgaben_df, personen_df)


=== Hungarian Algorithm ===
Execution time: 0.027 seconds
Found assignments: 8
Total score: 7999992.00


In [92]:
for result in hungarian_results:
    matching_ergebnisse.append({
        "teilaufgabe_id": result["teilaufgabe_id"],
        "teilaufgabe": result["teilaufgabe"],
        "projekt_id": result["projekt_id"],
        "kompetenz": result["kompetenz"],
        "person_id": result["person_id"],
        "name": result["name"],
        "zugewiesener_aufwand": result["zugewiesener_aufwand"]
    })

print(f"\nHungarian Algorithm Results:")
print(f"Found {len(matching_ergebnisse)} assignments")
print(f"Execution time: {hungarian_time:.3f} seconds")


Hungarian Algorithm Results:
Found 8 assignments
Execution time: 0.045 seconds


In [93]:
# Display results summary
print(f"\n" + "="*50)
print("RESULTS SUMMARY")
print("="*50)
print(f"Total assignments: {len(matching_ergebnisse)}")
print(f"Total workload assigned: {sum(item['zugewiesener_aufwand'] for item in matching_ergebnisse):.1f} hours")
print(f"Total workload needed: {teilaufgaben_df['aufwand'].sum():.1f} hours")


RESULTS SUMMARY
Total assignments: 8
Total workload assigned: 19.5 hours
Total workload needed: 19.5 hours


In [94]:
# Show assignments by task
print(f"\nAssignments by task:")
for _, ta in teilaufgaben_df.iterrows():
    task_assignments = [item for item in matching_ergebnisse if item['teilaufgabe_id'] == ta['teilaufgabe_id']]
    assigned_workload = sum(item['zugewiesener_aufwand'] for item in task_assignments)
    print(f"  - {ta['bezeichnung']}: {assigned_workload:.1f}/{ta['aufwand']:.1f} hours assigned")


Assignments by task:
  - Entwurf CAD-Baugruppe: 3.0/3.0 hours assigned
  - Simulation FEM-Struktur: 2.5/2.5 hours assigned
  - Signalverarbeitung Sensor: 2.0/2.0 hours assigned
  - Entwicklung Python-Modul: 3.5/3.5 hours assigned
  - Statistische Auswertung: 2.0/2.0 hours assigned
  - Optische Analyse: 1.5/1.5 hours assigned
  - Datenbankmodellierung: 2.0/2.0 hours assigned
  - Thermodynamische Bewertung: 3.0/3.0 hours assigned


In [95]:
# Save results
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print(f"\nMatching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")


Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'
